In [1]:
import tensorflow as tf
import os
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
data_dir = '/local-scratch/localhome/mkhademi/BOLD5000_2.0/'
batch_size = 20
dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_coco.tfrecords']).batch(batch_size)
train_ds = dataset.take(1800//batch_size)
test_ds = dataset.skip(1800//batch_size)
test_ds = test_ds.take(200//batch_size)
train_ds = train_ds.shuffle(1800)

In [3]:
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72), dtype=tf.float32),
            'y': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'y_super': tf.io.FixedLenFeature(shape=(12), dtype=tf.int64),
            'y_common': tf.io.FixedLenFeature(shape=(90+12), dtype=tf.int64),
            'y_face': tf.io.FixedLenFeature(shape=(2), dtype=tf.int64),
        })
    return example['x'], example['y_face']
    
decoded = train_ds.map(tf_parse)
decoded_test = test_ds.map(tf_parse)

In [4]:
raw_example = next(iter(test_ds))
print(tf_parse(raw_example)[0].numpy().shape)
print(tf_parse(raw_example)[1].numpy().shape)
print(tf_parse(raw_example)[1].numpy().sum(axis=1))

(20, 71, 89, 72)
(20, 2)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [5]:
for x_batch, y_batch in decoded_test.take(1):
    print(x_batch, y_batch)

tf.Tensor(
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0.

In [6]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(71, 89, 72)),
    tf.keras.layers.Conv2D(4, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(8, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.5),
    # tf.keras.layers.Dense(1000, activation='sigmoid'),
    # tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(2),
    # tf.keras.layers.Softmax()
    ])
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 87, 4)         2596      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 34, 43, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 41, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 20, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 18, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 9, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1008)              0

In [7]:
model.compile(
    # loss='mse',
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
    # optimizer=keras.optimizers.RMSprop(),
    optimizer='adam',
    # metrics=[tf.keras.metrics.MeanSquaredError()],
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    # metrics=[tf.keras.metrics.CategoricalCrossentropy(from_logits=True)]
)
model.fit(decoded, epochs=60)

Epoch 1/60
90/90 [==============================] - 6s 17ms/step - loss: 90.5809 - categorical_accuracy: 0.6228
Epoch 2/60
90/90 [==============================] - 4s 16ms/step - loss: 51.7486 - categorical_accuracy: 0.6344
Epoch 3/60
90/90 [==============================] - 4s 17ms/step - loss: 54.9745 - categorical_accuracy: 0.6428
Epoch 4/60
90/90 [==============================] - 4s 16ms/step - loss: 19.8828 - categorical_accuracy: 0.6328
Epoch 5/60
90/90 [==============================] - 4s 17ms/step - loss: 9.8657 - categorical_accuracy: 0.6567
Epoch 6/60
90/90 [==============================] - 4s 16ms/step - loss: 17.7010 - categorical_accuracy: 0.6683
Epoch 7/60
90/90 [==============================] - 4s 17ms/step - loss: 8.2337 - categorical_accuracy: 0.6544
Epoch 8/60
90/90 [==============================] - 4s 17ms/step - loss: 6.8449 - categorical_accuracy: 0.6717
Epoch 9/60
90/90 [==============================] - 4s 16ms/step - loss: 4.7950 - categorical_accuracy: 0.6

In [8]:
loss, accuracy = model.evaluate(decoded_test)

10/10 [==============================] - 4s 24ms/step - loss: 1.0615 - categorical_accuracy: 0.6450


In [9]:
model.add(tf.keras.layers.Softmax())
for x_batch, y_batch in decoded_test.take(1):
    print(model(x_batch))

tf.Tensor(
[[0.74894935 0.25105065]
 [0.5574328  0.4425673 ]
 [0.3480626  0.6519374 ]
 [0.34052703 0.65947294]
 [0.54096884 0.45903116]
 [0.15519138 0.8448086 ]
 [0.86336243 0.13663757]
 [0.55161756 0.4483824 ]
 [0.5735887  0.4264113 ]
 [0.00793173 0.9920683 ]
 [0.6495843  0.3504157 ]
 [0.9261085  0.07389154]
 [0.8678593  0.13214067]
 [0.13361202 0.86638796]
 [0.5992439  0.4007561 ]
 [0.18910058 0.8108994 ]
 [0.02422627 0.97577375]
 [0.2621735  0.73782647]
 [0.74302685 0.25697318]
 [0.3220107  0.67798924]], shape=(20, 2), dtype=float32)


In [ ]:
batch_size = 20
dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_common.tfrecords']).batch(batch_size)
train_ds = dataset.take((2135+400)//batch_size)
test_ds = dataset.skip((2135+400)//batch_size)
test_ds = test_ds.take(329//batch_size)
train_ds = train_ds.shuffle(2135+400)

In [ ]:
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72), dtype=tf.float32),
            'y': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'y_super': tf.io.FixedLenFeature(shape=(12), dtype=tf.int64),
            'y_common': tf.io.FixedLenFeature(shape=(90+12), dtype=tf.int64),
            'y_face': tf.io.FixedLenFeature(shape=(2), dtype=tf.int64),
        })
    return example['x'], example['y_super']
    
decoded = train_ds.map(tf_parse)
decoded_test = test_ds.map(tf_parse)

In [ ]:
raw_example = next(iter(test_ds))
print(tf_parse(raw_example)[0].numpy().shape)
print(tf_parse(raw_example)[1].numpy().shape)
print(tf_parse(raw_example)[1].numpy().sum(axis=1))

In [ ]:
for x_batch, y_batch in decoded_test.take(1):
    print(x_batch, y_batch)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(71, 89, 72)),
    tf.keras.layers.Conv2D(4, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(8, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.5),
    # tf.keras.layers.Dense(1000, activation='sigmoid'),
    # tf.keras.layers.Dense(100, activation='sigmoid'),
    tf.keras.layers.Dense(12),
    # tf.keras.layers.Softmax()
    ])
 
model.summary()

In [ ]:
model.compile(
    # loss='mse',
    # loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    # optimizer=keras.optimizers.RMSprop(),
    optimizer='adam',
    # metrics=[tf.keras.metrics.MeanSquaredError()],
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    # metrics=[tf.keras.metrics.CategoricalCrossentropy(from_logits=True)]
)
model.fit(decoded, epochs=60)

In [ ]:
loss, accuracy = model.evaluate(decoded_test)

In [ ]:
for x_batch, y_batch in decoded_test.take(1):
    print(tf.keras.activations.sigmoid(model(x_batch)))